# Creating a Decorator

Although Vanguard has a number of out-of-the-box decorators to allow for advanced Gaussian processes techniques, one might need something more specialist. Luckily, decorators in Vanguard are designed to be as extensible as possible. This walkthrough will explain how to create a new decorator to shuffle the input data passed to a controller.

In [ ]:
# sphinx ignore

import sys

sys.path.append("../..")

In [ ]:
from gpytorch.kernels import RBFKernel
from gpytorch.likelihoods import FixedNoiseGaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import torch

from vanguard.base import GPController
from vanguard.decoratorutils import Decorator, process_args, wraps_class
from vanguard.optimise import SmartOptimiser
from vanguard.uncertainty import GaussianUncertaintyGPController

## Recapping Python Decorators

In Python, a decorator is a function which returns another function.  Consider the following function:

In [ ]:
def is_py_file(file_path):
    """Returns True if file_path is a Python file."""
    return file_path.endswith(".py")


is_py_file("foo.py"), is_py_file("bar.js")

In [ ]:
# sphinx expect AttributeError

is_py_file(42)

In [ ]:
def check_string(func):
    """Check that the input is a string."""
    def inner_function(*args):
        for arg in args:
            if not isinstance(arg, str):
                raise TypeError("All inputs must be strings.")
        return func(*args)
    return inner_function

The decorator can then be applied in the following fashion:

In [ ]:
# sphinx expect TypeError

@check_string  # equivalent to: is_py_file = check_string(is_py_file)
def is_py_file(file_path):
    """Returns True if file_path is a Python file."""
    return file_path.endswith(".py")


is_py_file("foo.py"), is_py_file("bar.js")

is_py_file(42)

Sometimes it is helpful for a decorator to accept some arguments to adjust its behaviour. In this case, the function in question just needs to return a *decorator*:

In [ ]:
def check_type(t):
    """Check that the input is of a certain type."""
    def decorator(func):
        def inner_function(*args):
            for arg in args:
                if not isinstance(arg, t):
                    raise TypeError(f"All inputs must be of type {t}.")
            return func(*args)
        return inner_function
    return decorator

In [ ]:
# sphinx expect TypeError

@check_type(str)  # equivalent to: is_py_file = check_type(str)(is_py_file)
def is_py_file(file_path):
    """Returns True if file_path is a Python file."""
    return file_path.endswith(".py")


is_py_file("foo.py"), is_py_file("bar.js")

is_py_file(42)

## Creating a Decorator: Shuffling Inputs

Consider the following function:

In [ ]:
def consistent_shuffle(*arrays, seed=None):
    """Shuffle all arrays into the same order, to maintain consistency."""
    rng = np.random.RandomState(seed=seed)
    indices = np.arange(len(arrays[0]))
    rng.shuffle(indices)

    shuffled_arrays = [array[indices] for array in arrays]
    return shuffled_arrays

In [ ]:
x = np.array([1, 2, 3, 4, 5])
y = np.array([1, 4, 9, 16, 25])

In [ ]:
consistent_shuffle(x, y, seed=1)

In [ ]:
process_args(GPController.__init__, None, x, y, RBFKernel, mean_class=ConstantMean, y_std=0.1,
             likelihood_class=FixedNoiseGaussianLikelihood,
             marginal_log_likelihood_class=ExactMarginalLogLikelihood, optimiser_class=torch.optim.Adam,
             smart_optimiser_class=SmartOptimiser)

In [ ]:
class ShuffleDecorator(Decorator):
    """Shuffles input data."""
    def __init__(self, **kwargs):
        super().__init__(framework_class=GPController, required_decorators={}, **kwargs)

    def _decorate_class(self, cls):

        class InnerClass(cls):
            """An inner class."""
            def __init__(self, *args, **kwargs):
                all_parameters_as_kwargs = process_args(super().__init__, *args, **kwargs)
                all_parameters_as_kwargs.pop("self")  # this needs to be removed

                old_train_x = all_parameters_as_kwargs.pop("train_x")
                old_train_y = all_parameters_as_kwargs.pop("train_y")
                old_y_std = all_parameters_as_kwargs.pop("y_std")  # pop to avoid duplication

                if isinstance(old_y_std, (float, int)):
                    old_y_std = np.ones_like(old_train_x) * old_y_std

                new_train_x, new_train_y, new_y_std = consistent_shuffle(old_train_x, old_train_y, old_y_std)

                super().__init__(train_x=new_train_x, train_y=new_train_y, y_std=new_y_std,
                                 **all_parameters_as_kwargs)

        return InnerClass

The decorator can now be applied to a controller class in one of two ways. The latter is recommended for readability and extension.

In [ ]:
ShuffledGPController = ShuffleDecorator()(GPController)


@ShuffleDecorator()
class ShuffledGPController(GPController):
    """Shuffles inputs to the controller."""
    pass

In [ ]:
print(ShuffledGPController.__name__)
print(ShuffledGPController.__doc__)

In [ ]:
class ShuffleDecorator(Decorator):
    """Shuffles input data."""
    def __init__(self, **kwargs):
        super().__init__(framework_class=GPController, required_decorators={}, **kwargs)

    def _decorate_class(self, cls):

        @wraps_class(cls)
        class InnerClass(cls):
            """An inner class."""
            def __init__(self, *args, **kwargs):
                all_parameters_as_kwargs = process_args(super().__init__, *args, **kwargs)
                all_parameters_as_kwargs.pop("self")  # this needs to be removed

                old_train_x = all_parameters_as_kwargs.pop("train_x")
                old_train_y = all_parameters_as_kwargs.pop("train_y")
                old_y_std = all_parameters_as_kwargs.pop("y_std")  # pop to avoid duplication

                if isinstance(old_y_std, (float, int)):
                    old_y_std = np.ones_like(old_train_x) * old_y_std

                new_train_x, new_train_y, new_y_std = consistent_shuffle(old_train_x, old_train_y, old_y_std)

                super().__init__(train_x=new_train_x, train_y=new_train_y, y_std=new_y_std,
                                 **all_parameters_as_kwargs)

        return InnerClass

In [ ]:
@ShuffleDecorator()
class ShuffledGPController(GPController):
    """Shuffles inputs to the controller."""
    pass


print(ShuffledGPController.__name__)
print(ShuffledGPController.__doc__)

In [ ]:
class ShuffleDecorator(Decorator):
    """Shuffles input data."""
    def __init__(self, seed=None, **kwargs):
        super().__init__(framework_class=GPController, required_decorators={}, **kwargs)
        self.seed = seed

    def _decorate_class(self, cls):
        seed = self.seed

        @wraps_class(cls)
        class InnerClass(cls):
            """An inner class."""
            def __init__(self, *args, **kwargs):
                all_parameters_as_kwargs = process_args(super().__init__, *args, **kwargs)
                all_parameters_as_kwargs.pop("self")  # this needs to be removed

                old_train_x = all_parameters_as_kwargs.pop("train_x")
                old_train_y = all_parameters_as_kwargs.pop("train_y")
                old_y_std = all_parameters_as_kwargs.pop("y_std")  # pop to avoid duplication

                if isinstance(old_y_std, (float, int)):
                    old_y_std = np.ones_like(old_train_x) * old_y_std

                new_train_x, new_train_y, new_y_std = consistent_shuffle(old_train_x, old_train_y, old_y_std, seed=seed)

                super().__init__(train_x=new_train_x, train_y=new_train_y, y_std=new_y_std,
                                 **all_parameters_as_kwargs)

        return InnerClass

Note the defining of the intermediate value `seed`, before entering `InnerClass`. This is necessary because within the scope of `InnerClass`, `self` no longer refers to the decorator instance.

In [ ]:
@ShuffleDecorator()
class ShuffledGaussianUncertaintyGPController(GaussianUncertaintyGPController):
    """Shuffles inputs to the controller."""
    pass

To acknowledge that these methods are not expected to affect the behaviour of the decorator, they must be explicitly ignored:

In [ ]:
@ShuffleDecorator(ignore_methods={'predict_at_point', '_get_additive_grad_noise', '_noise_transform',
                                  '_append_constant_to_infinite_generator'})
class ShuffledGaussianUncertaintyGPController(GaussianUncertaintyGPController):
    """Shuffles inputs to the controller."""
    pass

In [ ]:
class ShuffleDecorator(Decorator):
    """Shuffles input data."""
    def __init__(self, seed=None, additional_params_to_shuffle=(), **kwargs):
        if additional_params_to_shuffle:
            kwargs["ignore_methods"] = set(kwargs["ignore_methods"]) | {"__init__"}

        super().__init__(framework_class=GPController, required_decorators={}, **kwargs)

        self.seed = seed
        self.params_to_shuffle = {"train_x", "train_y", "y_std"} | set(additional_params_to_shuffle)

    def _decorate_class(self, cls):
        seed = self.seed
        params_to_shuffle = self.params_to_shuffle

        @wraps_class(cls)
        class InnerClass(cls):
            """An inner class."""
            def __init__(self, *args, **kwargs):
                all_parameters_as_kwargs = process_args(super().__init__, *args, **kwargs)
                all_parameters_as_kwargs.pop("self")  # this needs to be removed

                array_for_reference = all_parameters_as_kwargs["train_x"]

                pre_shuffled_args = [all_parameters_as_kwargs.pop(param) for param in params_to_shuffle]
                pre_shuffled_args_as_arrays = [np.ones_like(array_for_reference) * arg if isinstance(arg, (float, int))
                                               else arg for arg in pre_shuffled_args]
                shuffled_args = consistent_shuffle(*pre_shuffled_args_as_arrays, seed=seed)

                shuffled_params_as_kwargs = dict(zip(params_to_shuffle, shuffled_args))

                super().__init__(**shuffled_params_as_kwargs, **all_parameters_as_kwargs)

        return InnerClass

There are a few changes to unpack here; take note of the following:

* If a user passes `additional_params_to_shuffle`, then it can be assumed that they have properly checked `__init__`, and it can be automatically ignored by the decorator.
* The popping and array-converting of parameters now needs to be less constrained, and done more programmatically.

In [ ]:
ignore_methods = {'_get_posterior_over_fuzzy_point_in_eval_mode', '__init__', '_sgd_round', '_process_x_std',
                  '_set_requires_grad', 'predict_at_point', '_get_additive_grad_noise', '_noise_transform',
                  '_append_constant_to_infinite_generator'}


@ShuffleDecorator(seed=1, additional_params_to_shuffle={"train_x_std"}, ignore_methods=ignore_methods)
class ShuffledGaussianUncertaintyGPController(GaussianUncertaintyGPController):
    """Shuffles inputs to the controller."""
    pass

There are plenty of other ways in which `ShuffleDecorator` can be improved or made more extendable, but the concepts are more or less the same.

In [ ]:
train_x = np.array([1, 2, 3, 4, 5])
train_x_std = np.array([0.01, 0.02, 0.03, 0.04, 0.05])
train_y = np.array([1, 4, 9, 16, 25])
y_std = np.array([0.02, 0.04, 0.06, 0.08, 0.1])

In [ ]:
controller = ShuffledGaussianUncertaintyGPController(train_x, train_x_std, train_y, y_std,
                                                     kernel_class=RBFKernel, mean_class=ConstantMean,
                                                     likelihood_class=FixedNoiseGaussianLikelihood,
                                                     marginal_log_likelihood_class=ExactMarginalLogLikelihood,
                                                     optimiser_class=torch.optim.Adam)

In [ ]:
print(controller.train_x.T)
print(controller.train_x_std.T)
print(controller.train_y.T)
print(controller._y_variance.T)